# Lineare Regression

## Daten laden & kennenlernen
Die Daten für diese Übung basieren auf [diesem Kaggle Datensatz](https://www.kaggle.com/datasets/abhishek14398/salary-dataset-simple-linear-regression)

**Lade die Daten und mache dich damit vertraut**

In [ ]:
import pandas as pd
df = pd.read_csv("03_Salary.csv")

## Lineare Regression modellieren
Bei einer **linearen Regression** möchten wir mit einer numerischen Variable eine andere numerische Variable vorhersagen.

Eine lineare Regression ergibt ein **Modell**. Das ist ein fancy Begriff für eine Gleichung oder **Funktion** $f(x)$, welche eine Variable $x$ als input nimmt und den Wert der Variable $y = f(x)$ **vorhersagt**.

Diese Funktion hat eine festgelegt Form: $f(x) = m*x+b$.

- Die Funktion stellt einer **Gerade** da (daher die Bezeichnung *linear*)
- $m$ beschreibt die **Steigung** der Funktion, also wie viel sich der Wert unserer Zielvariable $y$ verändert, wenn wir unsere Inputvariable (auch **Faktor** oder **Prädiktor** genannt) um eine Einheit verändern. Auf englisch wird dieser Wert oft **slope** genannt.
- $b$ beschreibt den $y$-Wert wenn $x=0$. Wir kennen das aus der Schule vielleicht noch als *Y-Achsen-Schnittpunkt*, weil die Gerade an diesem Punkt die Y-Achse schneidet). Auf englisch wird dieser Wert oft **intercept** genannt
- $x$ ist der Wert unseres Prädiktors für welchen wir eine Vorhersage über $y$ machen wollen. Dort können wir jeden beliebigen Wert eintragen und das Modell wird eine Vorhersage machen.

In Python kann man Lineare Regressionen auf viele Arten berechnen, sowohl "manuell" als auch mit vorgefertigten Funktionen. Dafür nutzen wir `scikit-learn` auch `sklearn` genannt.



In [ ]:
# Wir importieren die nötige Funktion
from sklearn.linear_model import LinearRegression

Wir bauen ein lineares Regressionsmodell. Dabei nutzen wir die Arbeitserfahrung als Pädiktor und das Gehalt als Zielvariable.

In [ ]:
X = pd.DataFrame(df.YearsExperience) # Inputs sind üblicherweise eine 2-D Matrix, also transformieren wir unsere einzelne Spalte zu einem DataFrame
y = df.Salary

model = LinearRegression()
model.fit(X, y)

# Get the results
slope = model.coef_[0]
intercept = model.intercept_

print(f"Modell: f(x) = {int(slope)} * x + {int(intercept)}")

**Versuche zu interpretieren, was dieses Modell aussagt**

Antwort:
- Ohne Arbeitserfahrung (also für $x=0$) sagt unser Modell ein Einsteigsgehalt von knapp 25.000€ voraus (das erkennen wir aus dem **intercept** $b$)
- Für jedes weitere Berufsjahr steigt das erwartete Gehalt um knapp 9.500€ 🚀 (das erkennen wir aus dem **Slope** $m$)

Wie viel werden wir laut unserem Modell also nach 10 Jahren verdienen?

Dazu können wir ganz einfach Werte in die Formel einsetzen.

In [ ]:
years_experience = 10
expected_salary = slope * years_experience + intercept

round(expected_salary,1)

Wir können auch das Modellobjekt nutzen um gleich mehrere Vorhersagen damit zu machen. Hier beispielsweise die Vorhersage für jedes der ersten 10 Berufsjahre.

In [ ]:
# Predict values
y_pred = model.predict(pd.DataFrame({"YearsExperience":[0,1,2,3,4,5,6,7,8,9,10]}))
print(f"Predicted values: {y_pred.round(1)}")

**Erscheinen euch diese Vorhersagen realistisch?**

# Modellqualität bewerten
Bevor wir unser Modell nutzen, um in eine Gehaltsverhandlung zu treten, sollten wir verifizieren, wie belastbar dessen Vorhersagen wirklich sind.

Dafür gibt es im wesentlichen zwei Methoden.

## Visuelle Inspektion
Wir können schauen, wie nah die von uns modellierte Gerade an den tatsächlich beobachteten Punkten ist.

In [ ]:
import matplotlib.pyplot as plt

So sind unsere historischen Daten verteilt

In [ ]:
ax = df.plot(x="YearsExperience", y="Salary", kind="scatter")
ax.set_xlim(0)
ax.set_ylim(0)
plt.show();

Wir können nun die Gerade darüberlegen

In [ ]:
ax = df.plot(x="YearsExperience", y="Salary", kind="scatter")
plt.axline((0, intercept), slope=slope, color='salmon', label=f'y = {int(intercept)} * x + {int(slope)}')

ax.set_xlim(0, 12)
ax.set_ylim(0, 150000)
plt.legend()
plt.show();

Wir erkennen, dass das Modell visuell tatsächlich recht gut passt. Die Gerade führt mehr oder weniger durch die Mitte der Verteilung und es gibt keinen x-Bereich, an welchem sie stark von den beobachteten Werten abweicht.

Eine **wichtige Einschränkung** ist, dass wir keine Daten für weniger als 1 Jahr Berufserfahrung und mehr als 10 Jahre Berufserfahrung beobachtet haben. Das Modell kann zwar auch in diesen Bereichen Vorhersagen tätigen, allerdings sollten wir dabei skeptisch sein, weil diese nicht historisch belegt sind.

## Empirische Bewertung
Bei linearen Regressionen wird oft der $R^2$-Wert herangezogen, um zu bewerten, ob ein Modell gut ist oder nicht, bzw. besser als ein anderes.

$R^2$ gibt an wie viel der Varianz der Beobachtungen durch das Modell *erklärt* werden. Je höher dieser Wert, desto besser das Modell. Etwas intuitiver ausgedrückt vergleichen wir die gesamte Abweichung, wenn wir einfach immer deren Mittelwert raten würden mit gesamten Abweichung wenn wir den vom Modell vorgeschlagenen Wert raten. Wenn das Modell gut ist, dann sinkt letztere Abweichung und der $R^2$ steigt.

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
predicted_salary = model.predict(pd.DataFrame(df.YearsExperience))
true_salary = df.Salary

r2_score(true_salary, predicted_salary)

**Interpretation**: Der $R^2$ Wert ist nahe 1, das heißt die Variabilität im Gehalt lässt sich fast vollständig durch die Berufserfahrung erklären.

## Schlussfolgerung - Können wir dem Modell trauen?
Wir haben sowohl visuell als auch empirisch gezeigt, dass das Modell gut passt. Allerdings ist es sehr wichtig zu betonen, dass das Modell nur zu **beobachteten Daten** gut passt und Vorhersagen nur für solche Personen sinnvoll sind, für die der Datensatz **repräsentativ** ist.

Wenn also der Datensatz in einem bestimmten Land erhoben wurde oder nur für eine bestimmte Branche dann repräsentiert das Modell auch nur die Gehaltsentwicklung dieses Landes oder dieser Branche.

Da wir nicht genau wissen, woher die Daten stammen und für welche Personen sie repräsentativ sind, können wir uns trotz der guten Ergebnisse leider nicht darauf verlassen.